In [285]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import polygon
from shapely.validation import make_valid
from shapely.validation import explain_validity
import fiona
import rtree
import pyproj


In [131]:
# File paths
z_usfs_th = '../data/source/usfs/S_USA.Activity_TimberHarvest.gdb.zip'
z_usfs_hft = '../data/source/usfs/S_USA.Activity_HazFuelTrt_PL.gdb.zip'
calfire_th = '../data/source/calfire/ds816/ds816.gdb'
calfire_ntmp = '../data/source/calfire/CAL_FIRE_Nonindustrial_Timber_Management_Plans_and_Notices_TA83.geojson'
z_calfire_rx = '../data/source/calfire/fire20_1.gdb.zip'
calfire_calmapper = '../data/source/calfire/FuelTreatments_CALFIRE22_1_public.gdb.zip'
z_census_states = '../data/source/census/tl_2020_us_state.zip'
z_census_counties = '../data/source/census/tl_2021_us_county.zip'

In [117]:
# Geodataframes

## Raw
usfs_th_df = gpd.read_file(z_usfs_th) # Forest Service timber harvest
usfs_hft_df = gpd.read_file(z_usfs_hft) # Forest Service hazardous fuel treatment
calfire_th_df = gpd.read_file(calfire_th) # CalFire timber harvest
calfire_ntmp_df = gpd.read_file(calfire_ntmp) # CalFire non-industrial timber management plans
calfire_rx_df = gpd.read_file(z_calfire_rx) # CalFire prescribed burns
calfire_calmapper_df = gpd.read_file(calfire_calmapper) # CalFire CALMAPPER (California Management Activity Project Planning and Event Reporter)
states_df = gpd.read_file(z_census_states)
counties_df = gpd.read_file(z_census_counties)

## Select only Calif. projects from USFS sets
usfs_th_ca_df = usfs_th_df.loc[usfs_th_df['STATE_ABBR'] == 'CA'].copy()
usfs_hft_ca_df = usfs_hft_df.loc[usfs_hft_df['STATE_ABBR'] == 'CA'].copy()

## Select only Calif. state and county shapes
ca_df = states_df[states_df['STUSPS'] == 'CA'].copy()
ca_counties_df = counties_df[counties_df['STATEFP'] == '06'].copy()


# TO DO
## [DONE] GOTTA VERIFY RX BECAUSE PREVIOUS URL RETURNED 404
## [DONE] GET calmapper set



In [286]:
tester = gpd.read_file(calfire_th) # CalFire timber harvest
# tester['APPROVED'] = pd.to_datetime(calfire_th_df['APPROVED'].str[:4], format='%Y', errors='coerce')
tester['APPROVED'] = pd.to_datetime(tester['APPROVED'].str[:4], format='%Y', errors='coerce')
tester['COMPLETED'] = pd.to_datetime(tester['COMPLETED'].str[:4], format='%Y', errors='coerce')


In [287]:
tester['MATHS'] = tester['COMPLETED'] - tester['APPROVED']
tester[['APPROVED', 'COMPLETED', 'MATHS']]

,APPROVED,COMPLETED,MATHS
0,2012-01-01,2015-01-01,1096 days
1,2012-01-01,2015-01-01,1096 days
2,2012-01-01,2015-01-01,1096 days
3,2011-01-01,2015-01-01,1461 days
4,2011-01-01,2015-01-01,1461 days
...,...,...,...
74188,2021-01-01,NaT,NaT
74189,2021-01-01,NaT,NaT
74190,2021-01-01,NaT,NaT
74191,2021-01-01,NaT,NaT


In [289]:
# RENAME/MUNGE COLUMNS TO MATCH ACROSS DATASETS

usfs_th_ca_df.rename(columns = {'SUID':'ID', 'ACTIVITY_NAME':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FY_PLANNED':'FY', 'FY_COMPLETED':'COMPLETED', 'OWNERSHIP_DESC':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_th_ca_df['FY'] = pd.to_datetime(usfs_th_ca_df['FY'], format='%Y', errors='coerce')
usfs_th_ca_df['COMPLETED'] = pd.to_datetime(usfs_th_ca_df['COMPLETED'], format='%Y', errors='coerce')

usfs_hft_ca_df.rename(columns = {'SUID':'ID', 'ACTIVITY':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FISCAL_YEAR_PLANNED':'FY', 'FISCAL_YEAR_COMPLETED':'COMPLETED', 'FS_UNIT_NAME':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_hft_ca_df['FY'] = pd.to_datetime(usfs_hft_ca_df['FY'], format='%Y', errors='coerce')
usfs_hft_ca_df['COMPLETED'] = pd.to_datetime(usfs_hft_ca_df['COMPLETED'], format='%Y', errors='coerce')

calfire_th_df.rename(columns = {'SILVI_1':'SILV1', 'SILVI_CAT':'SILV2', 'LANDOWNER':'OWNER', 'GIS_ACRES':'ACRES'}, inplace = True)
calfire_th_df['ID'] = calfire_th_df.index + 1 #For some reason the OBJECTID column wasn't showing up but it appears to be a sequential count, so this hacky index workaround should work.
calfire_th_df['FY'] = pd.to_datetime(calfire_th_df['APPROVED'].str[:4], format='%Y', errors='coerce')
calfire_th_df['COMPLETED'] = pd.to_datetime(calfire_th_df['COMPLETED'].str[:4], format='%Y', errors='coerce')

calfire_ntmp_df.rename(columns = {'OBJECTID':'ID', 'SILVI_1':'SILV1', 'SILVI_CAT':'SILV2', 'LANDOWNER':'OWNER', 'GIS_ACRES':'ACRES'}, inplace = True)
calfire_ntmp_df['FY'] = pd.to_datetime(calfire_ntmp_df['APPROVED'].str[:4], format='%Y', errors='coerce')
calfire_ntmp_df['COMPLETED'] = pd.to_datetime(calfire_ntmp_df['CANCELLED'].str[:4], format='%Y', errors='coerce') #Clarke uses cancelled as completed here. I think we should probalby just say that completion is not tracked here ... which seems to be the case.


In [290]:
usfs_th_ca_clean_df = usfs_th_ca_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
usfs_hft_ca_clean_df = usfs_hft_ca_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
calfire_th_clean_df = calfire_th_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
calfire_ntmp_clean_df = calfire_ntmp_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]

In [300]:
calfire_th_onlycompleted_df = calfire_th_df[calfire_th_df['PLAN_STAT'] == 'Completed'].copy()
calfire_th_onlycompleted_clean_df = calfire_th_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(calfire_th_df.index))
print(len(calfire_th_onlycompleted_clean_df.index))

74193
74193


In [297]:
usfs_th_ca_nodups_df = usfs_th_ca_df.drop_duplicates(['ID', 'GIS_ACRES']).copy()
usfs_th_ca_nodups_clean_df = usfs_th_ca_nodups_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_th_df.index))
print(len(usfs_th_nodups_clean_df.index))

787170
700882


In [298]:
usfs_hft_ca_nodups_df = usfs_hft_ca_df.drop_duplicates(['ID', 'GIS_ACRES']).copy()
usfs_hft_ca_nodups_clean_df = usfs_hft_ca_nodups_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_hft_df.index))
print(len(usfs_hft_nodups_df.index))

480353
269126


In [121]:
# TODO
## Sort out which columns map to which in Clarke's
## Map intensity
## Spot check .head() resutls against QGIS?

In [272]:
usfs_th_ca_nodups_df.rename(columns = {'SUID':'ID', 'ACTIVITY_NAME':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FY_PLANNED':'FY', 'FY_COMPLETED':'COMPLETED', 'OWNERSHIP_DESC':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_th_ca_nodups_clean_df = usfs_th_ca_nodups_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_th_ca_nodups_df.index))
print(len(usfs_th_ca_nodups_clean_df.index))

68514
68514


In [157]:
usfs_hft_ca_nodups_df.rename(columns = {'SUID':'ID', 'ACTIVITY':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FISCAL_YEAR_PLANNED':'FY', 'FISCAL_YEAR_COMPLETED':'COMPLETED', 'FS_UNIT_NAME':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_hft_ca_nodups_clean_df = usfs_hft_ca_nodups_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_hft_ca_nodups_clean_df.index))

40416


In [126]:
print(usfs_hft_ca_nodups_clean_df['ACRES'].sum())
print(usfs_th_ca_nodups_clean_df['ACRES'].sum())

3215359.0
2229559.6


In [173]:
calfire_th_df['ID'] = calfire_th_df.index + 1 #For some reason the OBJECTID column wasn't showing up but it appears to be a sequential count, so this hacky index workaround should work.
calfire_th_df['FY'] = calfire_th_df['APPROVED'].str[:4]
calfire_th_df['COMPLETED'] = calfire_th_df['COMPLETED'].str[:4]
calfire_th_df.rename(columns = {'SILVI_1':'SILV1', 'SILVI_CAT':'SILV2', 'LANDOWNER':'OWNER', 'GIS_ACRES':'ACRES'}, inplace = True)
calfire_th_clean_df = calfire_th_df[['ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES']]
print(len(calfire_th_clean_df.index))

74193


In [174]:
calfire_ntmp_df['FY'] = calfire_ntmp_df['APPROVED'].str[:4]
calfire_ntmp_df['COMPLETED'] = calfire_ntmp_df['CANCELLED'].str[:4] #Clarke uses cancelled as completed here. I think we should probalby just say that completion is not tracked here ... which seems to be the case.
calfire_ntmp_df.rename(columns = {'OBJECTID':'ID', 'SILVI_1':'SILV1', 'SILVI_CAT':'SILV2', 'LANDOWNER':'OWNER', 'GIS_ACRES':'ACRES'}, inplace = True)
calfire_ntmp_clean_df = calfire_ntmp_df[['ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES']]
print(len(calfire_ntmp_clean_df.index))

5905


In [260]:
calfire_th_ntmp_clean_df = pd.concat([calfire_th_clean_df, calfire_ntmp_clean_df])
calfire_th_ntmp_clean_df = calfire_th_ntmp_clean_df.fillna(value=np.nan)
calfire_th_ntmp_clean_df['FY'] = calfire_th_ntmp_clean_df['FY'].astype('float')
calfire_th_ntmp_clean_df['COMPLETED'] = calfire_th_ntmp_clean_df['COMPLETED'].astype('float')

usfs_th_hft_ca_dups_clean_df = pd.concat([usfs_hft_ca_dups_clean_df, usfs_th_ca_dups_clean_df])
usfs_th_hft_ca_dups_clean_df = usfs_th_hft_ca_dups_clean_df.fillna(value=np.nan)
usfs_th_hft_ca_dups_clean_df['FY'] = usfs_th_hft_ca_dups_clean_df['FY'].astype(float)
usfs_th_hft_ca_dups_clean_df['COMPLETED'] = usfs_th_hft_ca_dups_clean_df['COMPLETED'].astype(float)

In [175]:
# ACTIVITY ACRES (i.e., with multiple treatments on same terrain)

In [183]:
usfs_th_ca_df.rename(columns = {'SUID':'ID', 'ACTIVITY_NAME':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FY_PLANNED':'FY', 'FY_COMPLETED':'COMPLETED', 'OWNERSHIP_DESC':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_th_ca_dups_clean_df = usfs_th_ca_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_th_ca_dups_clean_df.index))
print(usfs_th_ca_dups_clean_df['ACRES'].sum())

usfs_hft_ca_df.rename(columns = {'SUID':'ID', 'ACTIVITY':'SILV1', 'TREATMENT_TYPE':'SILV2', 'FISCAL_YEAR_PLANNED':'FY', 'FISCAL_YEAR_COMPLETED':'COMPLETED', 'FS_UNIT_NAME':'OWNER', 'NBR_UNITS_ACCOMPLISHED':'ACRES'}, inplace = True)
usfs_hft_ca_dups_clean_df = usfs_hft_ca_df.loc[:, ('ID', 'SILV1', 'SILV2', 'FY', 'COMPLETED', 'OWNER', 'ACRES')]
print(len(usfs_hft_ca_dups_clean_df.index))
print(usfs_hft_ca_dups_clean_df['ACRES'].sum())

82435
4481875.2


In [261]:
usfs_th_hft_ca_dups_clean_df = pd.concat([usfs_hft_ca_dups_clean_df, usfs_th_ca_dups_clean_df])
print(len(usfs_th_hft_ca_dups_clean_df.index))
print(usfs_th_hft_ca_dups_clean_df['ACRES'].sum())

usfs_th_hft_ca_dups_clean_df['FY'] = usfs_th_hft_ca_dups_clean_df['FY'].astype(int)
usfs_th_hft_ca_dups_clean_84_to_pres_df = usfs_th_hft_ca_dups_clean_df[usfs_th_hft_ca_dups_clean_df['FY'] >= 1984]
print(usfs_th_hft_ca_dups_clean_84_to_pres_df['ACRES'].sum())

162881
7162369.400000001
6763546.500000001


In [270]:
usfs_th_hft_ca_dups_clean_84_to_pres_df = pd.concat([usfs_th_hft_ca_dups_clean_df, calfire_th_ntmp_clean])
usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_df = usfs_th_hft_ca_dups_calfire_th_ntmp_clean_df[usfs_th_hft_ca_dups_calfire_th_ntmp_clean_df['FY'] >= 1984]
print(len(usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_df.index))

usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_completed_df = usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_df[usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_df['COMPLETED'] >= 1984]
print(len(usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_completed_df.index))

usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_planned = usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_completed_df[usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_completed_df['COMPLETED'].isnull()]
print(len(usfs_th_hft_ca_dups_calfire_th_ntmp_clean_84_to_pres_planned.index))


# usfs_th_hft_ca_dups_calfire_th_ntmp_clean_df.dtypes

#%(@*!&%(*#^@)$(*&)@#%^
    
# # Filter to get planned activity acres
# # Data with intra/inter dups, that I am filtering to capture 
# # NAs in the year completed column (column 6). Thus, I am 
# # storing approved but NOT completed entries in a new dataframe
#     facts_calfire_1984to2019_dups_planned <- facts_calfire_1984to2019_dups %>% 
#       filter(!complete.cases(Completed))
    
# #### Back to no dups data sets    
    
# # 4. Merge federal/state data sets together;
# #    Take out inter-duplication of management events;
# #    Filter to 1984-2019;
# #    Bind all federal and state data sets together

    
# # Merge/bind FACTS:
#     bind_facts <- bind_rows(facts_th_clean, facts_hft_clean, .id=NULL)
#     sum(bind_facts$Acres) ## 3,405,566
    
# # Remove inter-duplication across timber harvest and haz fuels
#     bind_facts_nodups <- bind_facts %>% 
#       distinct(ID, Acres, .keep_all = TRUE)
#     sum(bind_facts_nodups$Acres) ## 2,089,019 acres

# # Filter to 1984-2019 on the no duplicate data set
#     facts_1984to2019 <- bind_facts_nodups %>% 
#       filter(between(FiscalYear, 1984,2019))
#     sum(facts_1984to2019$Acres) #1,943,015 acres
    
# # Merge/bind CAL FIRE data sets:
#     bind_calfire <- bind_rows(calfire_thp_clean, calfire_ntmp_clean, .id=NULL)
#     bind_calfire <- transform(bind_calfire, FiscalYear = as.numeric(FiscalYear), Completed = as.numeric(Completed))
    
# # Filter to new time frame (note data begins 1991)    
#     calfire_1984to2019 <- bind_calfire %>% 
#       filter(between(FiscalYear, 1984,2019))
#     sum(calfire_1984to2019$Acres) #3,799,480 acres
    
# # Completed events
#     calfire_1984to2019_completed <- bind_calfire %>% 
#       filter(between(Completed, 1984,2019))
#     sum(calfire_1984to2019_completed$Acres) #2,786,925 acres
    
#     sum(calfire_thp_onlycompleted$ACRES) # 2,684,463

# # Bind FACTS and Cal-Fire data sets together 
# # (note: this will have years before 1984 for FACTS)
#     library(plyr)
#     facts_calfire <- rbind.fill(bind_facts_nodups, bind_calfire, .id=NULL)
#     head(facts_calfire)
#     dim(facts_calfire)
    
#     facts_calfire <- transform(facts_calfire, FiscalYear = as.numeric(FiscalYear), 
#                                Completed = as.numeric(Completed))

# # Filter to the time period 1984 to 2019
# # Note, I can still sort for completed or not in this data set
#     facts_calfire_1984to2019 <- facts_calfire %>% 
#       filter(between(FiscalYear, 1984,2019))  

224995
143283
0


In [229]:
# # Todo
# ## Fix geometries and add a column that notes if they've been fixed, remove NoneTypes, then try this again.

# usfs_th_df_val = usfs_th_df 
# usfs_th_df_val['VALIDATED'] = ''

# for idx, row in usfs_th_df_val.iterrows():
#     g = row['geometry']
#     if isinstance(g, type(None)):
#         usfs_th_df_val.drop(index=idx)
#     elif g.is_valid == False:
#         usfs_th_df_val.iat[idx, 68] = make_valid(g)
#         usfs_th_df_val.iat[idx, 69] = 1
#     else:
#         usfs_th_df_val.iat[idx, 69] = 0
    

# # res_union = ca_df.overlay(usfs_th_df, how='union')
# # res_union

# # Done
# ## Select USFS projects in CA only
# ## Filter USFS sets to Calif. only

In [31]:
usfs_th_df.describe(include='all')

/Users/glevines/.local/share/virtualenvs/ca-wildfire-mitigation-ii_yUWW0/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
/Users/glevines/.local/share/virtualenvs/ca-wildfire-mitigation-ii_yUWW0/lib/python3.9/site-packages/pandas/core/dtypes/inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/Users/glevines/.local/share/virtualenvs/ca-wildfire-mitigation-ii_yUWW0/lib/python3.9/site-packages/pandas/core/dtypes/inference.py:384: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the l

,ADMIN_FOREST_CODE,ADMIN_REGION_CODE,ADMIN_FOREST_NAME,PROCLAIMED_FOREST_CODE,ADMIN_DISTRICT_NAME,ADMIN_DISTRICT_CODE,HOME_ORG,ACTIVITY_UNIT_ORG,SUID,FACTS_ID,...,ACCURACY,CRC_VALUE,UK,EDW_INSERT_DATE,ETL_MODIFIED,REV_DATE,GIS_ACRES,SHAPE_Length,SHAPE_Area,geometry
count,787170,787170,787170,787169,787170,787170,787170,787170,787170,787170,...,115406.000000,787170,0,787170,787170,405041,639633.000000,787170.000000,787170.000000,639633
unique,29,9,112,154,518,41,515,589,700882,467245,...,NaN,787168,0,3191,3191,34803,NaN,NaN,NaN,563313
top,05,06,NULL,0905,NULL,01,011405,011405,0511034990099000000,0613051901,...,NaN,000000009B5AFDC0,NaN,2021-01-19T02:56:41+00:00,2021-01-19T02:56:41+00:00,1999-11-30T00:00:00+00:00,NaN,NaN,NaN,(POLYGON ((-113.50994244399999 45.157269046000...
freq,79664,174060,35709,31829,26152,109071,12322,12322,28,318,...,NaN,2,NaN,29123,29123,14647,NaN,NaN,NaN,22
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.971050,NaN,NaN,NaN,NaN,NaN,39.377271,0.017575,0.000014,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1696.574235,NaN,NaN,NaN,NaN,NaN,151.157865,0.026946,0.000061,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,10.753000,0.006657,0.000002,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,21.846000,0.014060,0.000007,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.000000,NaN,NaN,NaN,NaN,NaN,41.020000,0.022363,0.000016,NaN


In [27]:
# s = usfs_th_df.is_valid

# with open("is_valid.txt", "a") as o:
#     o.write(str(s))

for idx, row in usfs_th_df.iterrows():
    g = row['geometry']
    with open('is_valid.txt', 'a') as o:
        if isinstance(g, type(None)):
            o.write(str(idx) + ', NoneType\n')
        elif g.is_valid == False:
            make_valid(g)
            o.write(str(idx) + ', ' + str(make_valid(g)) + '\n')
        else:
            o.write(str(idx) + ', already valid\n')

    # print(row['geometry'])
    # with open("is_valid.txt", "a") as o:
    #     if isinstance(row['geometry'], type(None)):
    #         o.write("NoneType")
    #     else:
    #         o.write(str(idx) + ", " + str(row['geometry'].is_valid))